mtcars 데이터셋(mtcars.csv)의 qsec 컬럼을 최소최대 척도(Min-Max Scale)로 변환한 후 0.5보다 
큰 값을 가지는 레코드 수를 구하시오.

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [21]:
df= pd.read_csv('./mtcars.csv')
scaler=MinMaxScaler()
new=scaler.fit_transform(np.array(df['qsec']).reshape(-1,1))
print((new>0.5).sum())

# new

9


고객 3,500명에 대한 학습용 데이터(y_train.csv, X_train.csv)를 이용하여 성별예측 모형을 만든 
후, 이를 평가용 데이터(X_test.csv)에 적용하여 얻은 2,482명 고객의 성별 예측값(남자일 확률)을 
다음과 같은 형식의 CSV 파일로 생성하시오.(제출한 모델의 성능은 ROC-AUC 평가지표에 따라 
채점) 

In [33]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,recall_score,f1_score
x_train,y_train=pd.read_csv('./X_train.csv',encoding='cp949'),pd.read_csv('./y_train.csv',encoding='cp949')
x_test=pd.read_csv('./X_test.csv',encoding='cp949')
encoder=LabelEncoder()


a=encoder.fit_transform(x_train['주구매상품'])
b=encoder.fit_transform(x_train['주구매지점'])
x_train['주구매상품']=a
x_train['주구매지점']=b



pd.isna(x_train).sum()
x_train.drop(['환불금액'],axis=1,inplace=True)
x_train.duplicated().sum()
x_train=x_train.iloc[:,[1,2,5,6,7,8]]
def outlier(x):
    x=np.array(x)
    q1=np.percentile(x,25)
    q3=np.percentile(x,75)
    iqr=q3-q1
    low=q1-(1.5*iqr)
    high=q3+(1.5*iqr)
    x[np.where((x>high)| (x<low))]=x.mean()

    return x
for i in range(len(x_train.columns)):
    x_train.iloc[:,i]=outlier(x_train.iloc[:,i])
    

scaler=MinMaxScaler()
x_scaled=scaler.fit_transform(x_train)

df=pd.DataFrame(x_scaled,columns=x_train.columns)
df['주구매상품']=a
df['주구매지점']=b
df['gender']=y_train['gender']
(df['gender']==1).sum()/len(df)


0.376

In [24]:
x_train,x_valid,y_train,y_valid=train_test_split(df,df['gender'] ,test_size=0.2,shuffle=True,stratify=df['gender'])
y_train.shape

(2800,)

In [25]:
model=LogisticRegression()
model.fit(x_train,y_train)


LogisticRegression()

In [30]:
pred=model.predict(x_valid)
f1_score(pred,y_valid)


1.0

In [34]:
x_test

,cust_id,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,3500,70900400,22000000,4050000.0,골프,부산본점,13,1.461538,0.789474,26
1,3501,310533100,38558000,48034700.0,농산물,잠실점,90,2.433333,0.369863,3
2,3502,305264140,14825000,30521000.0,가공식품,본 점,101,14.623762,0.083277,3
3,3503,7594080,5225000,NaN,주방용품,부산본점,5,2.000000,0.000000,47
4,3504,1795790,1411200,NaN,수산품,청량리점,3,2.666667,0.125000,8
...,...,...,...,...,...,...,...,...,...,...
2477,5977,82581500,23976000,NaN,골프,부산본점,8,1.750000,0.642857,40
2478,5978,480000,480000,NaN,섬유잡화,광주점,1,1.000000,0.000000,0
2479,5979,260003790,25750000,NaN,남성 캐주얼,본 점,19,3.736842,0.915493,18
2480,5980,88991520,18120000,NaN,육류,본 점,5,3.600000,0.444444,60
